# Yemen Accessibility Pre-Process Graph

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

import matplotlib.pyplot as plt

In [30]:
# Get reference to GOSTNets
sys.path.append(r'../../../../GOSTnets')
import GOSTnets as gn

In [31]:
from GOSTnets.load_osm import *

## Roads were edited in QGIS
Roads to be rehabilitated were either classified as such, or created. Then the nodes and edges were saved in QGIS.

In [32]:
new_node_gdf = gpd.read_file('G_largest_cleaned_yemen4_single_nodes.shp')

In [33]:
new_node_gdf

,node_ID,y,x,geometry
0,1,15.453828,44.189288,POINT (44.18929 15.45383)
1,2,15.334732,44.212201,POINT (44.21220 15.33473)
2,5,13.804760,43.251596,POINT (43.25160 13.80476)
3,6,15.050154,50.489797,POINT (50.48980 15.05015)
4,7,13.072445,44.870837,POINT (44.87084 13.07244)
...,...,...,...,...
148501,9999058,NaN,NaN,POINT (43.87632 13.84178)
148502,9999007,NaN,NaN,POINT (44.60309 14.34715)
148503,new_obj_99991,13.190016,44.313079,POINT (44.31308 13.19002)
148504,new_obj_99993,13.222139,44.339780,POINT (44.33978 13.22214)


In [34]:
new_edge_gdf = gpd.read_file('G_largest_cleaned_yemen4_single_edge.shp')

In [35]:
new_edge_gdf.loc[new_edge_gdf['one_way'] > 0, 'one_way'] = True
new_edge_gdf.loc[new_edge_gdf['one_way'] == 0.0, 'one_way'] = False

In [36]:
new_edge_gdf.loc[new_edge_gdf['one_way']==False]

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry
0,1,110147,201859,False,810908760,legitimate,unclassified,edge_201859,0.061234,None,"LINESTRING (44.18929 15.45383, 44.18931 15.45438)"
1,1,144649,201873,False,810908766,legitimate,unclassified,edge_201873,0.155478,None,"LINESTRING (44.18929 15.45383, 44.18784 15.45386)"
4,2,16447,8846,False,47929605,legitimate,residential,edge_8846,0.058515,None,"LINESTRING (44.21220 15.33473, 44.21275 15.33475)"
5,5,951,99289,False,463039122,legitimate,residential,edge_99289,0.080134,None,"LINESTRING (43.25160 13.80476, 43.25154 13.805..."
6,5,93665,99290,False,463039123,legitimate,residential,edge_99290,0.028315,None,"LINESTRING (43.25160 13.80476, 43.25183 13.80487)"
...,...,...,...,...,...,...,...,...,...,...,...
205805,9999055,9999058,9999060,False,9999060,rehab,unclassified,9999060,3.833844,afiosh,"LINESTRING (43.87632 13.84178, 43.87627 13.842..."
205806,137995,19664,22823601,False,22823601,legitimate,track,22823601,0.161791,None,"LINESTRING (44.29819 16.55001, 44.29800 16.550..."
205807,new_obj_99991,new_obj_99992,91919191,False,91919191,rehab,unclassified,91919191,1.432734,Karabah_soq_arrabooa,"LINESTRING (44.31308 13.19002, 44.31325 13.190..."
205808,new_obj_99992,new_obj_99993,91919192,False,91919192,rehab,unclassified,91919192,5.068312,Karabah_soq_arrabooa,"LINESTRING (44.31852 13.19905, 44.31855 13.199..."


In [37]:
new_edge_gdf[new_edge_gdf['Type'] == 'rehab']

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry
178,170,147350,91987,False,436184635,rehab,trunk,edge_91987,0.977847,al_abr_to_seiyun,"LINESTRING (48.15696 15.75085, 48.15600 15.750..."
191,177,2173,153551,False,634019827,rehab,unclassified,edge_153551,0.330114,hajdah_rasian,"LINESTRING (43.75779 13.69880, 43.75772 13.698..."
414,400,30149,112346,False,499373690,rehab,unclassified,edge_112346,1.654408,mnar_anes,"LINESTRING (44.16420 14.59870, 44.16424 14.598..."
1278,1216,new_obj_413,28266,True,116670474,rehab,secondary,edge_28266,0.349498,taiz_to_al_turbah,"LINESTRING (43.96759 13.58206, 43.96779 13.582..."
1384,1319,136500,157794,False,653615856,rehab,unclassified,edge_157794,2.170767,al_zone_jarmah_track,"LINESTRING (43.77195 15.05488, 43.77208 15.055..."
...,...,...,...,...,...,...,...,...,...,...,...
205803,9999056,3267,9999057,False,9999057,rehab,unclassified,9999057,0.299337,afiosh,"LINESTRING (43.87574 13.83813, 43.87582 13.838..."
205804,9999058,3267,153610,False,634031153,rehab,unclassified,edge_153610,0.225436,afiosh,"LINESTRING (43.87632 13.84178, 43.87636 13.841..."
205805,9999055,9999058,9999060,False,9999060,rehab,unclassified,9999060,3.833844,afiosh,"LINESTRING (43.87632 13.84178, 43.87627 13.842..."
205807,new_obj_99991,new_obj_99992,91919191,False,91919191,rehab,unclassified,91919191,1.432734,Karabah_soq_arrabooa,"LINESTRING (44.31308 13.19002, 44.31325 13.190..."


In [38]:
new_edge_gdf[new_edge_gdf.duplicated(['osm_id'])]

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry
293,287,92141,187385,False,788317836,legitimate,residential,edge_187385,0.136270,None,"LINESTRING (44.98393 12.98537, 44.98376 12.985..."
304,299,110328,34982,True,121026285,legitimate,secondary,edge_34982,0.143315,None,"LINESTRING (44.21629 15.24922, 44.21609 15.249..."
335,331,26138,11927,False,90335278,legitimate,secondary,edge_11927,2.853896,None,"LINESTRING (44.17324 14.80015, 44.17312 14.800..."
443,425,35150,176382,False,776530048,legitimate,residential,edge_176382,0.026691,None,"LINESTRING (44.40407 14.50967, 44.40409 14.509..."
468,445,new_obj_5635,35253,True,122710288,destination_destruction,tertiary,edge_35253,0.039367,None,"LINESTRING (44.97667 12.86166, 44.97634 12.86143)"
...,...,...,...,...,...,...,...,...,...,...,...
205797,9999050,48630,154301,False,636402717,legitimate,unclassified,edge_154301,0.464283,None,"LINESTRING (43.89716 13.95542, 43.89680 13.955..."
205799,9999052,8763,154261,False,636387285,rehab,unclassified,edge_154261,3.250770,afiosh,"LINESTRING (43.89233 13.90785, 43.89222 13.907..."
205801,9999055,72591,153604,False,634031150,rehab,unclassified,edge_153604,2.008134,afiosh,"LINESTRING (43.88170 13.86275, 43.88154 13.863..."
205804,9999058,3267,153610,False,634031153,rehab,unclassified,edge_153610,0.225436,afiosh,"LINESTRING (43.87632 13.84178, 43.87636 13.841..."


In [39]:
# QAQC
#new_edge_gdf[new_edge_gdf.duplicated(['key'])]

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry


In [40]:
new_edge_gdf[new_edge_gdf['Type'] == 'rehab']

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry
178,170,147350,91987,False,436184635,rehab,trunk,edge_91987,0.977847,al_abr_to_seiyun,"LINESTRING (48.15696 15.75085, 48.15600 15.750..."
191,177,2173,153551,False,634019827,rehab,unclassified,edge_153551,0.330114,hajdah_rasian,"LINESTRING (43.75779 13.69880, 43.75772 13.698..."
414,400,30149,112346,False,499373690,rehab,unclassified,edge_112346,1.654408,mnar_anes,"LINESTRING (44.16420 14.59870, 44.16424 14.598..."
1278,1216,new_obj_413,28266,True,116670474,rehab,secondary,edge_28266,0.349498,taiz_to_al_turbah,"LINESTRING (43.96759 13.58206, 43.96779 13.582..."
1384,1319,136500,157794,False,653615856,rehab,unclassified,edge_157794,2.170767,al_zone_jarmah_track,"LINESTRING (43.77195 15.05488, 43.77208 15.055..."
...,...,...,...,...,...,...,...,...,...,...,...
205803,9999056,3267,9999057,False,9999057,rehab,unclassified,9999057,0.299337,afiosh,"LINESTRING (43.87574 13.83813, 43.87582 13.838..."
205804,9999058,3267,153610,False,634031153,rehab,unclassified,edge_153610,0.225436,afiosh,"LINESTRING (43.87632 13.84178, 43.87636 13.841..."
205805,9999055,9999058,9999060,False,9999060,rehab,unclassified,9999060,3.833844,afiosh,"LINESTRING (43.87632 13.84178, 43.87627 13.842..."
205807,new_obj_99991,new_obj_99992,91919191,False,91919191,rehab,unclassified,91919191,1.432734,Karabah_soq_arrabooa,"LINESTRING (44.31308 13.19002, 44.31325 13.190..."


## Creates a new graph from the edited node and edge GDF

In [41]:
new_graph = gn.edges_and_nodes_gdf_to_graph(new_node_gdf, new_edge_gdf, geometry_tag = 'geometry', largest_G = True, add_missing_reflected_edges=True, oneway_tag='one_way')

In [42]:
# save graph for Yemen analysis notebooks
gn.save(new_graph, 'yemen_regular_graph_wo_speeds3', 'yemen_regular_graph_wo_speeds3', pickle = True, edges = False, nodes = False)